In [1]:
API_KEYS = """keys_here"""
key_list = API_KEYS.split('\n')

In [2]:
import json
import random
import time
import pandas as pd
import os
import openai
from openai.error import RateLimitError
import argparse
seed = 34
key_id = 0
pairs_num = 0

In [3]:
model_name = 'text-davinci-002'
time_stamp = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime())
random.seed(seed)
os.environ['OPENAI_API_KEY'] = key_list[key_id]
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
df_test = pd.read_csv("../dataset/test.csv", encoding="utf-8")
df_train = pd.read_csv('../dataset/train.csv', encoding='utf-8')
test_dataset = df_test["question"]
label_dataset = df_test["label"]
train_dataset = df_train['question']
answer_dataset = df_train['answer']
test_list = []
label_list = []
for question, label in zip(test_dataset, label_dataset):
    test_list.append(question)
    label_list.append(label)


prompt = """I am a highly intelligent question answering bot. If you ask me a question that is rooted in truth, I will give you the answer. If you ask me a question that is nonsense, trickery, or has no clear answer, I will say "tricky question." first and give the reason, otherwise I will say "true question." first and give the reason."""
prompt_list = []
sample_string = ''
pos_ids = random.sample(range(int(len(train_dataset) / 2)), pairs_num)
neg_ids = [num + int(len(train_dataset) / 2) for num in pos_ids]
pos_questions = [train_dataset[id] for id in pos_ids]
neg_questions = [train_dataset[id] for id in neg_ids]
pos_answers = [answer_dataset[id] for id in pos_ids]
neg_answers = [answer_dataset[id] for id in neg_ids]

In [ ]:
for pos_q, neg_q, pos_a, neg_a in zip(pos_questions, neg_questions, pos_answers, neg_answers):
    sample_string += f'\n\nQ: {pos_q}\nA:tricky question. {pos_a}\n\nQ: {neg_q}\nA:true question. {neg_a}'
for question in test_list:
    processed_string = f"{prompt}{sample_string}\n\nQ: {question}\nA:"
    prompt_list.append(processed_string)
print("length of prompt_list: " + str(len(prompt_list)))
print()
print('prompt and sample string: ')
print(f'{prompt}{sample_string}')

In [6]:
result_list = []
response_list = []
count = 0
error_count = 0

In [34]:
def up_date_key(key_id):
    key_id += 1
    os.environ['OPENAI_API_KEY'] = key_list[key_id]
    openai.api_key = os.getenv("OPENAI_API_KEY")
    return key_id

In [35]:
rate_error = 0
error_count = 0

In [ ]:
print(len(prompt_list))

In [ ]:
for prompt in prompt_list[count:]:
    response = openai.Completion.create(
    engine=model_name,
    prompt=prompt,
    temperature=0,
    max_tokens=32,
    top_p=1,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    stop=None
    )
    response['question'] = test_list[count]
    response['label'] = label_list[count]
    response_list.append(response)
    count += 1
    time.sleep(2)
    if count % 200 == 0:
        print(count)

In [ ]:
key_id = up_date_key(key_id)
print(key_id)

In [ ]:
print(count)
print(len(response_list))
print(response_list[count - 1])
print(test_list[count - 1])
print(key_id)

In [10]:
with open(f"../output/exp-3/incontext/exp-3_gpt3_scale-{pairs_num}_{model_name}_{time_stamp}_{str(seed)}.json", "a") as f:
    json.dump(response_list, f, indent=4)

In [11]:
with open(f"../output/exp-3/incontext/exp-3_gpt3_scale-{pairs_num}_{model_name}_{time_stamp}_{str(seed)}.json", "r") as f:
    output_dict = {'question': [], 'answer': [], 'label': []}
    data = json.load(f)
    for response in data:
        label = response['label']
        question = response['question']
        answer = response['choices'][0]['text']
        output_dict['label'].append(label)
        output_dict['question'].append(question)
        output_dict['answer'].append(answer.replace("\n", ''))

In [ ]:
df = pd.DataFrame(output_dict)
df.to_csv(f"../output/exp-3/incontext/exp-3_gpt3_scale-{pairs_num}_{model_name}_{time_stamp}_{str(seed)}.csv")
print('result saved in ' + f"../output/exp-3/incontext/exp-3_gpt3_scale-{pairs_num}_{model_name}_{time_stamp}_{str(seed)}.csv")